In [ ]:
import scraping as s
import util as u
%load_ext autoreload
%autoreload 2

import requests
from bs4 import BeautifulSoup

import json
import re

In [17]:
import datetime as dt

In [2]:
token = 'HTVFOfb4OTB04zvV4zfmtv8Oeu5cKecEHRA_aw4gi6-4KC2kaaszNQ6yIQxLGera'

## Use song-artist string as "key" in my database

## Billboard

### Workflow into Genius

**Cycle through weeks**

In [3]:
top_20_base_url = 'https://www.billboard.com/charts/country-songs'
album_base_url = 'https://genius.com/albums/'

Scrape a week's top-20 lists. 
- Check if song-artist entry already in 'song_artist_list'
- if not, "do_all_the_things"

Can't pickle dictionary "recursion limit reached". Use MongoDB instead

## MongoDB

In [4]:
from pymongo import MongoClient
from pymongo import InsertOne, DeleteOne, ReplaceOne

SSH tunnel in thru command line:

```ssh -NL 12345:localhost:27017 kbfreder@```

Also start Mongo from command line -- log onto AWS

In [5]:
client = MongoClient(port=12345)

In [6]:
client.list_database_names()

['admin', 'local', 'song_db']

In [7]:
db = client.song_db

In [9]:
db.list_collection_names()

['lyrics_backup', 'lyrics', 'song_db']

In [13]:
# col = db.get_collection('song_db')
# col = db.create_collection('lyrics')
col = db.get_collection('lyrics')

In [15]:
col.estimated_document_count()

1093

In [12]:
genre = 'country'

In [11]:
# only initialize these the first time
song_artist_list = []
album_url_list = []

In [31]:
first_date = dt.datetime(2013,1,26)
stop_date = dt.datetime(2008,1,1)

next_date = first_date
temp_dict = {}

while next_date > stop_date:
    next_url = next_date.strftime("%Y-%m-%d")
    url = f'{top_20_base_url}/{next_url}'
    print(f'Scraping charts for week of: {next_url}', end = '\r')

    new_list = s.scrape_top_20(url)
    
    for entry in new_list:
        if entry not in song_artist_list:
#             print("Adding new entry! ", entry)
            song_entry = s.get_song_entry(entry, genre)
            temp_dict[song_entry[0]] = song_entry[1]
            song_artist_list.append(entry)
            
            album_url = song_entry[2]
            if album_url not in album_url_list:
                album_url_list.append(album_url)
        
    if len(temp_dict) > 20:
        print("Performing an insert")
        col.insert_many(temp_dict.values())
        temp_dict = {}
        
    next_date -= dt.timedelta(days=7)


if len(temp_dict) != 0:
    col.insert_many(temp_dict.values())
    temp_dict={}

print ('Done')

Performing an insertweek of: 2012-12-22
Performing an insertweek of: 2012-11-03
Performing an insertweek of: 2012-10-06
Performing an insertweek of: 2012-08-25
Performing an insertweek of: 2012-07-07
Performing an insertweek of: 2012-05-26
Performing an insertweek of: 2012-03-24
Performing an insertweek of: 2012-02-11
Performing an insertweek of: 2011-12-17
Performing an insertweek of: 2011-10-29
Performing an insertweek of: 2011-09-17
Performing an insertweek of: 2011-07-23
Performing an insertweek of: 2011-06-04
Performing an insertweek of: 2011-04-23
Performing an insertweek of: 2011-03-05
Performing an insertweek of: 2011-01-22
Performing an insertweek of: 2010-12-04
Performing an insertweek of: 2010-10-09
Performing an insertweek of: 2010-08-21
Performing an insertweek of: 2010-07-10
Performing an insertweek of: 2010-05-22
Performing an insertweek of: 2010-04-03
Performing an insertweek of: 2010-02-20
Performing an insertweek of: 2010-01-09
Performing an insertweek of: 2009-11-07


In [33]:
len(temp_dict)

0

In [25]:
col.insert_many(temp_dict.values())

In [34]:
col.estimated_document_count()

3045

In [35]:
len(song_artist_list)

1964

In [36]:
len(album_url_list)

635

In [45]:
# Check for duplicates
len(list(col.aggregate ( [
    {'$group': {'_id': {"song":"$song", "artist":"$artist"}, "unique_ids": {"$addToSet": "$_id"}, "count": {"$sum": 1}}},
    {"$match": {"count": { "$gte": 2}}},
    { "$group": { '_id': None, "count": { "$sum": 1 } } }
    ])))


1

In [46]:
list(col.aggregate ( [
    {'$group': {'_id': {"song":"$song", "artist":"$artist"}, "unique_ids": {"$addToSet": "$_id"}, "count": {"$sum": 1}}},
    {"$match": {"count": { "$gte": 2}}}
    ]))

[{'_id': {'song': 'Home', 'artist': 'Blake Shelton'},
  'unique_ids': [ObjectId('5be60535a034d705272ad065'),
   ObjectId('5be5fd93a034d705272acd58'),
   ObjectId('5be5f8d7a034d705272acced')],
  'count': 3},
 {'_id': {'song': "Rockin' The Beer Gut", 'artist': 'Trailer Choir'},
  'unique_ids': [ObjectId('5be60328a034d705272acf8b'),
   ObjectId('5be602a9a034d705272acf5f')],
  'count': 2}]

In [47]:
# back-up collection
pipeline = [ {"$match": {}}, 
             {"$out": "lyrics_backup"},
]

col.aggregate(pipeline)

In [48]:
# Remove duplicates
agg = col.aggregate ( [
    {'$group': {'_id': {"song":"$song", "artist":"$artist"}, 
                "unique_ids": {"$addToSet": "$_id"}, 
                "count": {"$sum": 1}}},
    {"$match": {"count": { "$gte": 2}}}
    ])

response = []
for doc in agg:
    del doc["unique_ids"][0]
    for id in doc["unique_ids"]:
        response.append(id)

# Pick one of these
# col.delete_one({"_id": {"$in": response}})
col.delete_many({"_id": {"$in": response}})
 
 # Check that it worked - lengths should be equal now
col.estimated_document_count() #, len(song_artist_list)

3042

## Save stuff

In [29]:
import dill

dill.dump_session('Data/session_dumps/session_20181109.pkl')

TypeError: Cannot serialize socket object

In [49]:
u.pkl_this('Data/song_artist_list_country.pkl', song_artist_list)
u.pkl_this('Data/album_url_list_country.pkl', album_url_list)